In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Read reference scenario data
df_ref = pd.read_csv('data/reference_emission_trajectories/avg_global_1_5.csv').set_index('Index').T[['share_2020']]
df_ref.index=pd.to_numeric(df_ref.index)
df_ref.share_2020 = pd.to_numeric(df_ref.share_2020)/100


# Interpolating emisison values between datapoints

df_ref = df_ref.reindex(range(2005, 2101)).interpolate(method='linear')


# read company targets
df_target_og=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Targets_OG')
df_target_car=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Targets_car')
df_target_airline=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Targets_Airlines')

# Drop columns with name Unnamed - conversion problem
columns_to_drop = [col for col in df_target_og.columns if 'Unnamed' in col]
df_target_og.drop(columns=columns_to_drop, inplace=True)
columns_to_drop = [col for col in df_target_car.columns if 'Unnamed' in col]
df_target_car.drop(columns=columns_to_drop, inplace=True)
columns_to_drop = [col for col in df_target_airline.columns if 'Unnamed' in col]
df_target_airline.drop(columns=columns_to_drop, inplace=True)

# Convert relevant columns to numeric
numeric_columns=['base_year', 'target_year', 'scope_coverage', 'planned_reduction']
df_target_og[numeric_columns]=df_target_og[numeric_columns].apply(pd.to_numeric, errors='coerce')
df_target_car[numeric_columns]=df_target_car[numeric_columns].apply(pd.to_numeric, errors='coerce')
df_target_airline[numeric_columns]=df_target_airline[numeric_columns].apply(pd.to_numeric, errors='coerce')

#get all coompanies
car=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Car').set_index('Company Name')
airline=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Airlines').set_index('Company Name')
oil_gas=pd.read_excel('data/company_data/Data.xlsx', sheet_name='O&G').set_index('Company Name')
companies = pd.concat([car, airline, oil_gas])


#convert scope name to string
df_target_og['scope']=df_target_og['scope'].astype(str)
df_target_car['scope']=df_target_car['scope'].astype(str)
df_target_airline['scope']=df_target_airline['scope'].astype(str)

df_target_og_2030=df_target_og[df_target_og['target_type']!="net-zero"]
df_target_car_2030=df_target_car[df_target_car['target_type']!="net-zero"]
df_target_airline_2030=df_target_airline[df_target_airline['target_type']!="net-zero"]

#get average scope coverage
df_share_by_scope=pd.read_pickle('data/temp/scope_share.pkl')


In [3]:
df_target_airline[df_target_airline['Company Name'].str.contains('easyJet')]

,Company Name,Country,Continent,CDP_ID,target_type,scope,target_no,scope2_method,scope3_cat,base_year,target_year,scope_coverage,planned_reduction,abs_int,source,CDP ref,Classic Carbon Credits,Intent_removal,comment
38,easyJet,United Kingdom of Great Britain and Northern I...,Airline,5170,intermediate,"1,3",1,NaN,Category 3: Fuel-and-energy-related activities...,2019.0,2035.0,100.0,35.0,int,CDP 2023,Int 1,Unsure,NaN,NaN
39,easyJet,United Kingdom of Great Britain and Northern I...,Airline,5170,net-zero,2,1,market_based,NaN,2019.0,2050.0,100.0,78.0,abs,CDP 2023,NZ1,yes,yes,We will offset emissions that aren't addressab...
40,easyJet,United Kingdom of Great Britain and Northern I...,Airline,5170,net-zero,"1,3",2,NaN,Category 3: Fuel-and-energy-related activities...,2019.0,2050.0,100.0,78.0,abs,CDP 2023,NZ1,yes,yes,We will offset emissions that aren't addressab...


## Scope 1 & 2 Emissions

In [4]:
#convert scope 1&2 target to ambition score
def scope12_target_to_ts(group):
    


    company_name=group['Company Name'].unique()[0]

    #print(company_name)
   

    

    df_return=pd.DataFrame()

    #for scope 1 and 2
    for scope in ['1','2']:
        #print(company_name, "Scope ", scope)
        subset=group[group['scope'].str.contains(scope)==True]
        subset.sort_values('target_no', ascending=True, inplace=True)

        year_list=[2020]
        target_list=[1]
        target_coverage=[]

        #print(group)
   

        column_name = f'scope_'+scope+'_target_share'

        #if there is an entrance which is not "-"
        if len(subset)>0 and  group['scope'].iloc[0] != "-":
            for index, row in subset.iterrows():
            
                base_year = row['base_year']
                target_year = row['target_year']
                scope_coverage=row['scope_coverage']
                planned_reduction = 1 - row['planned_reduction']/100

                #solve equation system to get share in  target year given that 2020 equals to 100% 
                if pd.isna(target_year):
                    print(company_name)
                    A = np.array([[2020, 1,0,0], [2020,1,-1,0],[2050,1,0,-1],[0,0,0,-1]])
                    B = np.array([100,0,0,0])
                
                else:

                    A = np.array([[2020, 1,0,0], [base_year ,1,-1,0],[target_year,1,0,-1],[0,0,planned_reduction,-1]])
                    B = np.array([100,0,0,0])

                # Solve the system of equations
              
                solution = np.linalg.solve(A, B)
                

                target_share = solution[3]

                
                
            
                
                target_share=target_share/100
                scope_coverage=scope_coverage/100

                #Adjust the target share based on the share of emission within scope that is covered by target
                target_share=target_share*scope_coverage+(1-scope_coverage)

              

      
                #Append target year and share
                year_list.append(target_year)
                target_list.append(target_share)

                target_coverage.append(scope_coverage)

                


                

                df=pd.DataFrame({'year':year_list, 'target_share':target_list}).set_index('year')
                    # Create a range of years you want to interpolate
                
                
                years_to_interpolate = range(2020, 2051)
                
     
                
                # Linearly interpolate between target years and shares to get a share for every year
                df = df.reindex(years_to_interpolate).interpolate(method='linear')
        else:
            df=pd.DataFrame({'year':range(2020,2051), 'target_share':1}).set_index('year')
        


            
        
        #add relevant years of reference scenario (i.e. 1.5°C trajectory)
        df['reference']=df_ref[(df_ref.index>=min(df.index)) & (df_ref.index<=max(df.index))]['share_2020']*100

        df_return[column_name]=df['target_share']
        df_return['reference']=df['reference']
        
    

    
    return(df_return)


 

      
   
#apply function to all companies
df_og_scope12=df_target_og.groupby('Company Name').apply(scope12_target_to_ts)


df_car_scope12=df_target_car.groupby('Company Name').apply(scope12_target_to_ts)

df_airline_scope12=df_target_airline.groupby('Company Name').apply(scope12_target_to_ts)


#apply function to all intermediate targets
df_og_scope12_2030=df_target_og_2030.groupby('Company Name').apply(scope12_target_to_ts)

df_car_scope12_2030=df_target_car_2030.groupby('Company Name').apply(scope12_target_to_ts)

df_airline_scope12_2030=df_target_airline_2030.groupby('Company Name').apply(scope12_target_to_ts)

/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/913153491.py:115: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_og_scope12=df_target_og.groupby('Company Name').apply(scope12_target_to_ts)
/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/913153491.py:118: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_car_scope12=df_target_car.groupby('Company Name').apply(scope12_target

In [5]:
df_airline_scope12.index.get_level_values(0).unique()

Index(['Air Canada', 'Air France-KLM', 'All Nippon Airways',
       'American Airlines Group', 'Avianca Group', 'Azul Sa',
       'Cathay Pacific Airways Limited', 'Chorus Aviation',
       'Copa Holdings, S.A.', 'Delta Air Lines', 'Deutsche Lufthansa',
       'Finnair', 'Gol Linhas Aereas Inteligentes S.A.',
       'Grupo Aeromexico SAB de CV', 'Hawaiian Holding',
       'International Airlines', 'Japan Airlines',
       'Jetblue Airways Corporation', 'Korean Air',
       'Norwegian Air Shuttle ASA', 'PEGASUS HAVA TAŞIMACILIĞI A.Ş.',
       'Ryanair Holdings', 'Southwest Airlines', 'Turkish Airlines',
       'United Airlines Holdings', 'Wizz Air Holdings', 'easyJet'],
      dtype='object', name='Company Name')

In [6]:
scope3_categories=['Purchased goods and services', 'Capital goods', 'Fuel-and-energy-related activities (not included in Scope 1 or 2)', 'Upstream transportation and distribution', 'Waste generated in operations', 'Business travel', 'Employee commuting', 'Upstream leased assets', 'Downstream transportation and distribution', 'Processing of sold products', 'Use of sold products', 'End of life treatment of sold products', 'Downstream leased assets', 'Franchises', 'Investments']

#convert scope 3 target to ambition score
def scope3_target_to_ts(group):
    company_name=group['Company Name'].unique()[0]
    print(company_name)
    df_return=pd.DataFrame()

    # for scope 3
    for scope in ['3']:
        #for all scope 3 categories
        for category in scope3_categories:
            #print(company_name, "Scope ", scope)

            if category=='Fuel-and-energy-related activities (not included in Scope 1 or 2)':
                category='Fuel-and-energy-related activities'


            #only use category of interest
            subset=group[group['scope'].str.contains(scope) & ((group['scope3_cat'].str.contains(category))|(group['scope3_cat']=='all'))]


            
            #remove targets that are not relevant (i.e. in data targets like 1a, 1b, indicate that there is a arrgetaed target called 1 somewhere which includes both 1a and 1b)
            subset = subset[~subset.target_no.astype(str).str.contains('a')]
            subset = subset[~subset.target_no.astype(str).str.contains('b')]
            subset = subset[~subset.target_no.astype(str).str.contains('c')]
                         
            #sort by target number (chronoogically)
            subset.sort_values('target_no', ascending=True, inplace=True)

            year_list=[2020]
            target_list=[1]
            target_coverage=[]

            column_name = category
            
            #if there is an entrance
            if len(subset)>0:
                for index, row in subset.iterrows():


                    base_year = row['base_year']
                    target_year = row['target_year']
                    scope_coverage=row['scope_coverage']
                    planned_reduction = 1 - row['planned_reduction']/100

                    #solve equation system to get share in  target year given that 2020 equals to 100%
                    A = np.array([[2020, 1,0,0], [base_year ,1,-1,0],[target_year,1,0,-1],[0,0,planned_reduction,-1]])
                    B = np.array([100,0,0,0])
                    # Solve the system of equations
                    solution = np.linalg.solve(A, B)

                    target_share = solution[3]
                    target_share=target_share/100
                    scope_coverage=scope_coverage/100

                    #add target years and shares
                    year_list.append(target_year)
                    target_list.append(target_share)

                    target_coverage.append(scope_coverage)
                    
                    
                    
                   

                   # Create a range of years you want to interpolate

                    df=pd.DataFrame({'year':year_list, 'target_share':target_list}).set_index('year')
                        # Create a range of years you want to interpolate


                    years_to_interpolate = range(2020, 2051)



                        
                    # Linearly interpolate between target years and shares to get a share for every year
              
                    df = df.reindex(years_to_interpolate).interpolate(method='linear')
            else:
                
                #if there is no entrance for the category, we assume that emissions remain unchanged
                df=pd.DataFrame({'year':range(2020,2051), 'target_share':1}).set_index('year')



                
            #add relevant years of reference scenario (i.e. 1.5°C trajectory)
            df_ref
            df['reference']=df_ref[(df_ref.index>=min(df.index)) & (df_ref.index<=max(df.index))]['share_2020']*100

            df_return[column_name]=df['target_share']
            df_return['reference']=df['reference']
        
    
        
        df_ref
        df['reference']=df_ref[(df_ref.index>=min(df.index)) & (df_ref.index<=max(df.index))]['share_2020']*100

        df_return[column_name]=df['target_share']
        df_return['reference']=df['reference']
        
    
    return(df_return)



#apply function to all companies
df_og_scope3=df_target_og.groupby('Company Name').apply(scope3_target_to_ts)
df_car_scope3=df_target_car.groupby('Company Name').apply(scope3_target_to_ts)
df_airline_scope3=df_target_airline.groupby('Company Name').apply(scope3_target_to_ts)

df_og_scope3.rename(columns={'Fuel-and-energy-related activities':'Fuel-and-energy-related activities (not included in Scope 1 or 2)'}, inplace=True)
df_car_scope3.rename(columns={'Fuel-and-energy-related activities':'Fuel-and-energy-related activities (not included in Scope 1 or 2)'}, inplace=True)
df_airline_scope3.rename(columns={'Fuel-and-energy-related activities':'Fuel-and-energy-related activities (not included in Scope 1 or 2)'}, inplace=True)



#apply function to all intermediate targets
df_og_scope3_2030=df_target_og_2030.groupby('Company Name').apply(scope3_target_to_ts)
df_car_scope3_2030=df_target_car_2030.groupby('Company Name').apply(scope3_target_to_ts)
df_airline_scope3_2030=df_target_airline_2030.groupby('Company Name').apply(scope3_target_to_ts)


df_og_scope3_2030.rename(columns={'Fuel-and-energy-related activities':'Fuel-and-energy-related activities (not included in Scope 1 or 2)'}, inplace=True)
df_car_scope3_2030.rename(columns={'Fuel-and-energy-related activities':'Fuel-and-energy-related activities (not included in Scope 1 or 2)'}, inplace=True)
df_airline_scope3_2030.rename(columns={'Fuel-and-energy-related activities':'Fuel-and-energy-related activities (not included in Scope 1 or 2)'}, inplace=True)





BP
Bharat Petroleum Corporation
Canadian Natural Resources
ConocoPhillips
Cosmo Energy Holdings Co., Ltd.
DTE Energy Company
Devon Energy
ENEOS Holdings
EOG Resources
Ecopetrol
Enbridge
Eni
Equinor
Formosa Petrochemical
Idemitsu Kosan Co., Ltd.
Inpex
KazMunayGas National Company JCS
Kinder Morgan
Koç Holding
MOL Nyrt.
Marathon Petroleum
OMV Group
Occidental Petroleum
PJSC Lukoil
PKN Orlen
PTT
Petrobras
Petroliam Nasional Berhad (PETRONAS)
Pioneer Natural Resources
Repsol
SK Innovation Co Ltd
San Miguel Corp
Schlumberger
Shell
Suncor Energy
TC Energy
TotalEnergies
Valero Energy
Williams
YPF SA
BMW Group
Ferrari
Ford Motor
General Motors
Hino Motors, Ltd.
Honda Motor
Hyundai Motor
Isuzu Motors


/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/1698132809.py:111: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_og_scope3=df_target_og.groupby('Company Name').apply(scope3_target_to_ts)


KIA
Mahindra & Mahindra
Mazda Motor
Mercedes-Benz Group
Mitsubishi
NFI Group Inc.
Nissan Motor
PACCAR Inc
Renault
SUBARU CORPORATION
Suzuki Motor
Tata Motors
Toyota Motor
Volkswagen Group
Volvo Group
Air Canada
Air France-KLM
All Nippon Airways
American Airlines Group
Avianca Group
Azul Sa
Cathay Pacific Airways Limited
Chorus Aviation
Copa Holdings, S.A.
Delta Air Lines


/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/1698132809.py:112: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_car_scope3=df_target_car.groupby('Company Name').apply(scope3_target_to_ts)


Deutsche Lufthansa
Finnair
Gol Linhas Aereas Inteligentes S.A.
Grupo Aeromexico SAB de CV
Hawaiian Holding
International Airlines
Japan Airlines
Jetblue Airways Corporation
Korean Air
Norwegian Air Shuttle ASA
PEGASUS HAVA TAŞIMACILIĞI A.Ş.
Ryanair Holdings
Southwest Airlines
Turkish Airlines
United Airlines Holdings
Wizz Air Holdings
easyJet
BP
Canadian Natural Resources
ConocoPhillips
Cosmo Energy Holdings Co., Ltd.
DTE Energy Company
Devon Energy
ENEOS Holdings


...

/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/1698132809.py:122: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_og_scope3_2030=df_target_og_2030.groupby('Company Name').apply(scope3_target_to_ts)


NFI Group Inc.
Nissan Motor
PACCAR Inc
Renault
SUBARU CORPORATION
Suzuki Motor
Tata Motors
Toyota Motor
Volkswagen Group
Volvo Group
Air Canada
Air France-KLM
All Nippon Airways
American Airlines Group
Avianca Group


/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/1698132809.py:123: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_car_scope3_2030=df_target_car_2030.groupby('Company Name').apply(scope3_target_to_ts)


Azul Sa
Copa Holdings, S.A.
Delta Air Lines
Deutsche Lufthansa
Finnair
Gol Linhas Aereas Inteligentes S.A.
Grupo Aeromexico SAB de CV
Hawaiian Holding
International Airlines
Japan Airlines
Jetblue Airways Corporation
Korean Air
Norwegian Air Shuttle ASA
PEGASUS HAVA TAŞIMACILIĞI A.Ş.
Ryanair Holdings
Southwest Airlines
Turkish Airlines
United Airlines Holdings
Wizz Air Holdings
easyJet


/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/1698132809.py:124: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_airline_scope3_2030=df_target_airline_2030.groupby('Company Name').apply(scope3_target_to_ts)


In [7]:
#calculate difference between reference (1.5 degree trajectory) and target share

df_og_scope12['diff_1']=df_og_scope12['reference']-df_og_scope12['scope_1_target_share']
df_og_scope12['diff_2']=df_og_scope12['reference']-df_og_scope12['scope_2_target_share']

df_car_scope12['diff_1']=df_car_scope12['reference']-df_car_scope12['scope_1_target_share']
df_car_scope12['diff_2']=df_car_scope12['reference']-df_car_scope12['scope_2_target_share']

df_airline_scope12['diff_1']=df_airline_scope12['reference']-df_airline_scope12['scope_1_target_share']
df_airline_scope12['diff_2']=df_airline_scope12['reference']-df_airline_scope12['scope_2_target_share']


df_og_scope12_2030['diff_1']=df_og_scope12_2030['reference']-df_og_scope12_2030['scope_1_target_share']
df_og_scope12_2030['diff_2']=df_og_scope12_2030['reference']-df_og_scope12_2030['scope_2_target_share']

df_car_scope12_2030['diff_1']=df_car_scope12_2030['reference']-df_car_scope12_2030['scope_1_target_share']
df_car_scope12_2030['diff_2']=df_car_scope12_2030['reference']-df_car_scope12_2030['scope_2_target_share']

df_airline_scope12_2030['diff_1']=df_airline_scope12_2030['reference']-df_airline_scope12_2030['scope_1_target_share']
df_airline_scope12_2030['diff_2']=df_airline_scope12_2030['reference']-df_airline_scope12_2030['scope_2_target_share']


In [8]:
#calculate difference between reference (1.5 degree trajectory) and target share for scope 3
df_scope3_by_cat=pd.read_pickle('data/temp/scope3_by_cat.pkl').replace(np.nan, 0)

#calculate scopee 3 emissions ambition by weighting category amibitions with share of total emissions
def calculate_total(df_scope3, df_scope3_by_cat, cat):
    df_scope3['total'] = np.nan
    for row in df_scope3.index:
        
        for column in df_scope3.columns:
            

            total = 0
            for i in df_scope3_by_cat.index:
            
                total += df_scope3_by_cat.loc[i, cat] * df_scope3.loc[row, i]
        
            df_scope3.loc[row, 'total'] = total
    return df_scope3



# Call the function passing the dataframes
df_og_scope3 = calculate_total(df_og_scope3, df_scope3_by_cat,'OG')
df_car_scope3 = calculate_total(df_car_scope3, df_scope3_by_cat,'car')
df_airline_scope3=calculate_total(df_airline_scope3, df_scope3_by_cat,'Airlines')


df_og_scope3['diff_3']=df_og_scope3['reference']-df_og_scope3['total']
df_car_scope3['diff_3']=df_car_scope3['reference']-df_car_scope3['total']
df_airline_scope3['diff_3']=df_airline_scope3['reference']-df_airline_scope3['total']

df_og_scope3_2030 = calculate_total(df_og_scope3_2030, df_scope3_by_cat,'OG')
df_car_scope3_2030 = calculate_total(df_car_scope3_2030, df_scope3_by_cat,'car')
df_airline_scope3_2030=calculate_total(df_airline_scope3_2030, df_scope3_by_cat,'Airlines')

df_og_scope3_2030['diff_3']=df_og_scope3_2030['reference']-df_og_scope3_2030['total']
df_car_scope3_2030['diff_3']=df_car_scope3_2030['reference']-df_car_scope3_2030['total']
df_airline_scope3_2030['diff_3']=df_airline_scope3_2030['reference']-df_airline_scope3_2030['total']


df_scope3_by_cat


,car,OG,Airlines
Purchased goods and services,0.168074,0.052965,0.104937
Capital goods,0.015673,0.002256,0.039100
Fuel-and-energy-related activities (not included in Scope 1 or 2),0.003689,0.073438,0.480481
Upstream transportation and distribution,0.010740,0.029479,0.060037
Waste generated in operations,0.001383,0.002192,0.001315
Business travel,0.000456,0.000479,0.003528
Employee commuting,0.001553,0.000397,0.010461
Upstream leased assets,0.000265,0.003508,0.006380
Downstream transportation and distribution,0.005415,0.040737,0.005480
Processing of sold products,0.030173,0.049471,0.000000


In [9]:
len(companies.index)

90

In [10]:
#make dataframe with all company IDs ans index
index_df=list(df_target_og['Company Name'].unique())+list(df_target_car['Company Name'].unique())+list(df_target_airline['Company Name'].unique())
df_result=pd.DataFrame(columns=['CDP_ID','scope1_ambition','scope2_ambition','scope3_ambition'],index=index_df)

# Calculate the sum of the differences between reference and target share for each company (i.e. difference for each year to difference 2020-2050) for scope 1 ambition
og_sum = df_og_scope12.reset_index().groupby('Company Name')['diff_1'].sum().sort_values()
car_sum = df_car_scope12.reset_index().groupby('Company Name')['diff_1'].sum().sort_values()
airline_sum = df_airline_scope12.reset_index().groupby('Company Name')['diff_1'].sum().sort_values()

# Concatenate the two series
combined_sum = pd.concat([og_sum, car_sum, airline_sum])

# Add to dataframe 
df_result['scope1_ambition'] = combined_sum

#Calculate the sum of the differences between reference and target share for each company (i.e. difference for each year to difference 2020-2050) for scope 2 ambition
og_sum = df_og_scope12.reset_index().groupby('Company Name')['diff_2'].sum().sort_values()
car_sum = df_car_scope12.reset_index().groupby('Company Name')['diff_2'].sum().sort_values()
airline_sum = df_airline_scope12.reset_index().groupby('Company Name')['diff_2'].sum().sort_values()

combined_sum = pd.concat([og_sum, car_sum, airline_sum])

df_result['scope2_ambition'] = combined_sum

#Calculate the sum of the differences between reference and target share for each company (i.e. difference for each year to difference 2020-2050) for scope 3 ambition
og_sum = df_og_scope3.reset_index().groupby('Company Name')['diff_3'].sum().sort_values()
car_sum = df_car_scope3.reset_index().groupby('Company Name')['diff_3'].sum().sort_values()
airline_sum = df_airline_scope3.reset_index().groupby('Company Name')['diff_3'].sum().sort_values()

combined_sum = pd.concat([og_sum, car_sum,airline_sum])

df_result['scope3_ambition'] = combined_sum


df_result['CDP_ID'] = companies['CDP_ID']
df_result['Sector'] = companies['Sector']

In [11]:
#make dataframe with all company IDs ans index for intermeidate targets
index_df=list(df_target_og_2030['Company Name'].unique())+list(df_target_car_2030['Company Name'].unique())+list(df_target_airline_2030['Company Name'].unique())
df_result_2030=pd.DataFrame(columns=['CDP_ID','scope1_ambition','scope2_ambition','scope3_ambition'],index=index_df)


# Calculate the sum of the differences between reference and target share for each company (i.e. difference for each year to difference 2020-2030) for scope 1 ambition for intermeidate targets

og_sum = df_og_scope12_2030[df_og_scope12_2030.index.get_level_values(1)<2031].reset_index().groupby('Company Name')['diff_1'].sum().sort_values()
car_sum = df_car_scope12_2030[df_car_scope12_2030.index.get_level_values(1)<2031].reset_index().groupby('Company Name')['diff_1'].sum().sort_values()
airline_sum = df_airline_scope12_2030[df_airline_scope12_2030.index.get_level_values(1)<2031].reset_index().groupby('Company Name')['diff_1'].sum().sort_values()


# Concatenate the two series
combined_sum = pd.concat([og_sum, car_sum, airline_sum])

df_result_2030['scope1_ambition'] = combined_sum

# Calculate the sum of the differences between reference and target share for each company (i.e. difference for each year to difference 2020-2030) for scope 2 ambition for intermeidate targets
og_sum = df_og_scope12_2030[df_og_scope12_2030.index.get_level_values(1)<2031].reset_index().groupby('Company Name')['diff_2'].sum().sort_values()
car_sum = df_car_scope12_2030[df_car_scope12_2030.index.get_level_values(1)<2031].reset_index().groupby('Company Name')['diff_2'].sum().sort_values()
airline_sum = df_airline_scope12_2030[df_airline_scope12_2030.index.get_level_values(1)<2031].reset_index().groupby('Company Name')['diff_2'].sum().sort_values()

combined_sum = pd.concat([og_sum, car_sum, airline_sum])


df_result_2030['scope2_ambition'] = combined_sum


# Calculate the sum of the differences between reference and target share for each company (i.e. difference for each year to difference 2020-2030) for scope 2 ambition for intermeidate targets

og_sum = df_og_scope3_2030[df_og_scope3_2030.index.get_level_values(1)<2031].reset_index().groupby('Company Name')['diff_3'].sum().sort_values()
car_sum = df_car_scope3_2030[df_car_scope3_2030.index.get_level_values(1)<2031].reset_index().groupby('Company Name')['diff_3'].sum().sort_values()
airline_sum = df_airline_scope3_2030[df_airline_scope3_2030.index.get_level_values(1)<2031].reset_index().groupby('Company Name')['diff_3'].sum().sort_values()

combined_sum = pd.concat([og_sum, car_sum,airline_sum])

df_result_2030['scope3_ambition'] = combined_sum


df_result_2030['CDP_ID'] = companies['CDP_ID']
df_result_2030['Sector'] = companies['Sector']


In [12]:
#read emission values
scope1=pd.read_csv('data/CDP/scope1_2023.csv')
scope2=pd.read_csv('data/CDP/scope2_2023.csv')
scope3=pd.read_csv('data/CDP/scope3_2023.csv')

scope1=scope1[scope1.RowName=='Reporting year']
scope2=scope2[scope2.RowName=='Reporting year']

scope3['scope3_emissions']=pd.to_numeric(scope3['scope3_emissions'],errors='coerce')
scope3=scope3.groupby('Account number')['scope3_emissions'].sum()

emissions=pd.DataFrame(columns=['scope1','scope2','scope3'],index=scope1.index)
emissions['scope1']=scope1['scope1_emissions']
emissions['scope2']=scope2['scope2_location_based']
emissions['scope3']=scope3


# complete for Schlumberger from sustainability report
emissions.loc[16418,'scope2']=401000

In [13]:
#calculate share of emissions that is covered by intermediate targe for each company
def share_intermediate(group):
    #filter for intermediate targets
    group=group[group['target_type']=='intermediate']
    df_intermediate=pd.DataFrame(columns=['share'])
    group.scope=group.scope.astype(str)



    #for each scope
    for scope in ['1','2','3']:
        #filter for targets containting scope
        sub_group=group[group['scope'].str.contains(scope)==True]


        #use coverage of target if there is only one entry
        if len(sub_group)==1:
            df_intermediate.loc['scope'+scope]=sub_group['scope_coverage'].iloc[0]

        #if there are multiple intermediate targets, use the one with the highest coverage
        elif len(sub_group)>1:
            df_intermediate.loc['scope'+scope]=sub_group['scope_coverage'].max()
            
        #if there are no intermediate targets, set coverage to 0
        elif len(sub_group)==0:
            df_intermediate.loc['scope'+scope]=0

    return df_intermediate



df_coverage=df_target_og.groupby('Company Name').apply(share_intermediate)
df_coverage = pd.concat([df_coverage, df_target_car.groupby('Company Name').apply(share_intermediate)])
df_coverage = pd.concat([df_coverage, df_target_airline.groupby('Company Name').apply(share_intermediate)])




/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/1750766058.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_coverage=df_target_og.groupby('Company Name').apply(share_intermediate)
/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/1750766058.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_coverage = pd.concat([df_coverage, df_target_car.groupby('Company Name').a

In [14]:
#calculate share of emissions that is covered by intensity target for each company

def share_intensity(group):

    #filter for intensity intermediate targets
    group=group[(group['abs_int']=='int')&(group['target_type']=='intermediate')]
    df_intensity=pd.DataFrame(columns=['share'])
    group.scope=group.scope.astype(str)



    #for each scope
    for scope in ['1','2','3']:
        sub_group=group[group['scope'].str.contains(scope)==True]


        #use coverage of target if there is only one entry
        if len(sub_group)==1:
            df_intensity.loc['scope'+scope]=sub_group['scope_coverage'].iloc[0]

        #if there are multiple intermediate targets, use the one with the highest coverage
        elif len(sub_group)>1:
            df_intensity.loc['scope'+scope]=sub_group['scope_coverage'].max()
            
        #if there are no intermediate targets, set coverage to 0
        elif len(sub_group)==0:
            df_intensity.loc['scope'+scope]=0

    return df_intensity


df_intensity_coverage=df_target_og.groupby('Company Name').apply(share_intensity)
df_intensity_coverage = pd.concat([df_intensity_coverage, df_target_car.groupby('Company Name').apply(share_intensity)])
df_intensity_coverage = pd.concat([df_intensity_coverage, df_target_airline.groupby('Company Name').apply(share_intensity)])


/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/45816779.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_intensity_coverage=df_target_og.groupby('Company Name').apply(share_intensity)
/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38815/45816779.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_intensity_coverage = pd.concat([df_intensity_coverage, df_target_car.gr

In [15]:
#add coverage to result dataframe
df_result[['scope1_coverage','scope2_coverage','scope3_coverage']]=np.nan
for i in df_result.index:


    df_result.loc[i,'scope1_coverage']=df_coverage.loc[(i,'scope1'),'share']
    df_result.loc[i,'scope2_coverage']=df_coverage.loc[(i,'scope2'),'share']
    df_result.loc[i,'scope3_coverage']=df_coverage.loc[(i,'scope3'),'share']


    df_result.loc[i, 'scope1_intensity']=df_intensity_coverage.loc[(i,'scope1'),'share']
    df_result.loc[i, 'scope2_intensity']=df_intensity_coverage.loc[(i,'scope2'),'share']
    df_result.loc[i, 'scope3_intensity']=df_intensity_coverage.loc[(i,'scope3'),'share']




df_result_2030[['scope1_coverage','scope2_coverage','scope3_coverage']]=np.nan
for i in df_result_2030.index:
    company=df_result_2030.loc[i,'CDP_ID']

    df_result_2030.loc[i,'scope1_coverage']=df_coverage.loc[(i,'scope1'),'share']
    df_result_2030.loc[i,'scope2_coverage']=df_coverage.loc[(i,'scope2'),'share']
    df_result_2030.loc[i,'scope3_coverage']=df_coverage.loc[(i,'scope3'),'share']

    df_result_2030.loc[i, 'scope1_intensity']=df_intensity_coverage.loc[(i,'scope1'),'share']
    df_result_2030.loc[i, 'scope2_intensity']=df_intensity_coverage.loc[(i,'scope2'),'share']
    df_result_2030.loc[i, 'scope3_intensity']=df_intensity_coverage.loc[(i,'scope3'),'share']


In [16]:
df_result[['scope1_share','scope2_share','scope3_share']]=np.nan

#weight by scope share of total emissions
for sector in companies.Sector.unique():
    sector_rows = df_result[df_result['Sector'] == sector]
    df_result.loc[sector_rows.index, 'scope1_share'] = df_share_by_scope.loc[sector, 'Scope 1']
    df_result.loc[sector_rows.index, 'scope2_share'] = df_share_by_scope.loc[sector, 'Scope 2']
    df_result.loc[sector_rows.index, 'scope3_share'] = df_share_by_scope.loc[sector, 'Scope 3']

df_result_2030[['scope1_share','scope2_share','scope3_share']]=np.nan
for sector in companies.Sector.unique():
    sector_rows = df_result_2030[df_result_2030['Sector'] == sector]
    df_result_2030.loc[sector_rows.index, 'scope1_share'] = df_share_by_scope.loc[sector, 'Scope 1']
    df_result_2030.loc[sector_rows.index, 'scope2_share'] = df_share_by_scope.loc[sector, 'Scope 2']
    df_result_2030.loc[sector_rows.index, 'scope3_share'] = df_share_by_scope.loc[sector, 'Scope 3']



In [17]:
df_result=df_result.reset_index().set_index('CDP_ID')
df_result_2030=df_result_2030.reset_index().set_index('CDP_ID')

df_result['weighted_ambition']=np.nan
df_result['weighted_intermediate_coverage']=np.nan
df_result['weighted_intensity_coverage']=np.nan

df_result_2030['weighted_ambition']=np.nan
df_result_2030['weighted_intermediate_coverage']=np.nan
df_result_2030['weighted_intensity_coverage']=np.nan



#weight scope ambition by share of emissions
for i in df_result.index:
    print(i)

    #total_emission=emissions.loc[i].sum()
    df_result.loc[i,'weighted_ambition']=(df_result.loc[i,'scope1_ambition']*df_result.loc[i,'scope1_share']+df_result.loc[i,'scope2_ambition']*df_result.loc[i,'scope2_share']+df_result.loc[i,'scope3_ambition']*df_result.loc[i,'scope3_share'])
    df_result.loc[i, 'weighted_intermediate_coverage'] = (df_result.loc[i, 'scope1_coverage'] * df_result.loc[i, 'scope1_share'] +
                                                      df_result.loc[i, 'scope2_coverage'] * df_result.loc[i, 'scope2_share'] +
                                                      df_result.loc[i, 'scope3_coverage'] * df_result.loc[i, 'scope3_share']) 
    
    df_result.loc[i, 'weighted_intensity_coverage'] = (df_result.loc[i, 'scope1_intensity'] * df_result.loc[i, 'scope1_share'] +
                                                        df_result.loc[i, 'scope2_intensity'] * df_result.loc[i, 'scope2_share'] +
                                                        df_result.loc[i, 'scope3_intensity'] * df_result.loc[i, 'scope3_share'])
    

for i in df_result_2030.index: 
    df_result_2030.loc[i,'weighted_ambition']=df_result_2030.loc[i,'scope1_ambition']*df_result_2030.loc[i,'scope1_share']+df_result_2030.loc[i,'scope2_ambition']*df_result_2030.loc[i,'scope2_share']+df_result_2030.loc[i,'scope3_ambition']*df_result_2030.loc[i,'scope3_share']
   # df_result_2030.loc[i,'weighted_ambition']=(df_result_2030.loc[i,'scope1_ambition']*emissions.loc[i,'scope1']+df_result_2030.loc[i,'scope2_ambition']*emissions.loc[i,'scope2']+df_result_2030.loc[i,'scope3_ambition']*emissions.loc[i,'scope3'])/total_emission
    df_result_2030.loc[i, 'weighted_intermediate_coverage'] = (df_result_2030.loc[i, 'scope1_coverage'] * df_result_2030.loc[i, 'scope1_share'] + df_result_2030.loc[i, 'scope2_coverage'] * df_result_2030.loc[i, 'scope2_share'] + df_result_2030.loc[i, 'scope3_coverage'] * df_result_2030.loc[i, 'scope3_share']) 


16012
19257
2083
23132
14654
5634
3751
36606
20048
13649
5581
17929
2667
15297
5767
16418
14774
15669
22341
14802
10150
13870
21135
29901
4678
19304
20705
9134
849766
11043
6602
21845
8793
20949
36979
3889
1800
12343
5021
839615
19290
20309
866803
1932
7164
12263
8708
8566
6595
840836
10076
13363
18074
15634
9559
59325
11520
6855
58656
8362
14169
11267
18285
4408
19569
660
4657
21148
17420
429
22213
36707
550
9759
5170
2926
58720
44253
73994
31588
7581
49618
6332
22460
23144
59365
10289
9703
50174
427


In [18]:
df_result.to_pickle('data/temp/ambition.pkl')

df_result